In [1]:
# Import libraries
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Training data and Testing data switch the commented lines to produce valid submission test.
# app_train = pd.read_csv('../Input/application_train.csv')
data = pd.read_csv('../Input/application_train.csv')
app_train, app_test = train_test_split(data, test_size = 0.2, random_state = 12)
# Testing data
# app_test = pd.read_csv('../Input/application_test.csv')

In [3]:
# Label encode

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [4]:
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (246008, 243)
Testing Features shape:  (61503, 242)


In [8]:
train_labels = app_train['TARGET']
#comment out next 2 lines when creating submission
test_labels = app_test['TARGET'] 
app_test = app_test.drop(columns = ['TARGET'])

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Add the target back in
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (246008, 243)
Testing Features shape:  (61503, 242)


In [9]:
# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

In [10]:
from sklearn.preprocessing import MinMaxScaler, Imputer

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()
    
# Feature names
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Median impuation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0,1))

# Fit on the traing data
imputer.fit(train)

# Transform both the training and testing data
train = imputer.transform(train)
test = imputer.transform(app_test)

# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (246008, 242)
Testing data shape:  (61503, 242)


In [23]:
from sklearn.ensemble import GradientBoostingClassifier

grad_tree = GradientBoostingClassifier(n_estimators=100, learning_rate=1, max_depth=5, random_state=12)
grad_tree.fit(train, train_labels)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=12, subsample=1.0, verbose=0,
              warm_start=False)

In [24]:
predictions = grad_tree.predict_proba(test)[:,1]

In [25]:
from sklearn.metrics import roc_auc_score
score = roc_auc_score(test_labels, predictions)
print(score)

0.72166928021


In [14]:
# Make a submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = predictions

# Save to a csv file
submit.to_csv('../Submissions/grad_tree.csv', index = False)

,SK_ID_CURR,TARGET
0,100001,0.052642
1,100005,0.103284
2,100013,0.023380
3,100028,0.023449
4,100038,0.113946
